<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Hyperparameter Tuning with Keras Tuner**


Estimated time needed: **30** minutes


In this lab, you will learn how to set up Keras Tuner and prepare the environment for hyperparameter tuning. 

## Learning objectives: 
By the end of this lab, you will: 
- Install Keras Tuner and import the necessary libraries
- Load and preprocess the MNIST data set
- Define a model-building function that uses hyperparameters to configure the model architecture
- Set up Keras Tuner to search for the best hyperparameter configuration 
- Retrieve the best hyperparameters from the search and build a model with these optimized values

## Prerequisites: 
- Basic understanding of Python programming 
- Keras and TensorFlow installed


### Exercise 1: Install the Keras Tuner 

This exercise guides you through the initial setup for using Keras Tuner. You install the library, import necessary modules, and load and preprocess the MNIST data set, which will be used for hyperparameter tuning. 
1. **Install Keras Tuner:**
    - Use pip to install Keras Tuner
2. **Import necessary libraries:**
    - Import Keras Tuner, TensorFlow, and Keras modules
3. **Load and preprocess the MNIST data set:**
    - Load the MNIST data set.
    - Normalize the data set by dividing by 255.0.


In [3]:
!pip install tensorflow==2.16.2
!pip install keras-tuner==1.4.7
!pip install numpy<2.0.0



/bin/bash: line 1: 2.0.0: No such file or directory


#### Explanation: 
This code installs the necessary libraries using pip

- **TensorFlow**: Ensures compatibility with the Keras Tuner.
- **Keras Tuner**: The version used in this lab.
- **Numpy**: Ensures compatibility with the other installed packages.


In [4]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

#### Explanation: 
The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.


In [5]:
# Step 2: Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only



2025-02-10 14:38:10.352272: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 14:38:10.353380: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-10 14:38:10.356973: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-10 14:38:10.372507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-10 14:38:10.395943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

#### Explanation

This code imports the necessary libraries:

- **`keras_tuner`**: Used for hyperparameter tuning.
- **`Sequential`**: A linear stack of layers in Keras.
- **`Dense`**, **`Flatten`**: Common Keras layers.
- **`mnist`**: The MNIST dataset, a standard dataset for image classification.
- **`Adam`**: An optimizer in Keras.


In [6]:

# Step 3: Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


#### Explanation

This code loads the MNIST dataset and preprocesses it:

- **`mnist.load_data()`**: Loads the dataset, returning training and validation splits.
- **`x_train / 255.0`**: Normalizes the pixel values to be between 0 and 1.
- **`print(f'...')`**: Displays the shapes of the training and validation datasets.


### Exercise 2: Defining the model with hyperparameters 

In this exercise, you define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate. This function returns a compiled Keras model that is ready for hyperparameter tuning.

**Define a model-building function:**
- Create a function `build_model` that takes a `HyperParameters` object as input.
- Use the `HyperParameters` object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.


In [7]:
# Define a model-building function 

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


#### Explanation

This function builds and compiles a Keras model with hyperparameters:

- **`hp.Int('units', ...)`**: Defines the number of units in the Dense layer as a hyperparameter.
- **`hp.Float('learning_rate', ...)`**: Defines the learning rate as a hyperparameter.
- **`model.compile()`**: Compiles the model with the Adam optimizer and sparse categorical cross-entropy loss.


### Exercise 3: Configuring the hyperparameter search 

This exercise guides you through configuring Keras Tuner. You create a `RandomSearch` tuner, specifying the model-building function, the optimization objective, the number of trials, and the directory for storing results. The search space summary provides an overview of the hyperparameters being tuned. 

**Create a RandomSearch Tuner:**
- Use the `RandomSearch` class from Keras Tuner. 
- Specify the model-building function, optimization objective (validation accuracy), number of trials, and directory for storing results.


In [8]:
# Create a RandomSearch Tuner 

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


#### Explanation

This code sets up a Keras Tuner `RandomSearch`:

- **`build_model`**: The model-building function.
- **`objective='val_accuracy'`**: The metric to optimize (validation accuracy).
- **`max_trials=10`**: The maximum number of different hyperparameter configurations to try.
- **`executions_per_trial=2`**: The number of times to run each configuration.
- **`directory='my_dir'`**: Directory to save the results.
- **`project_name='intro_to_kt'`**: Name of the project for organizing results.

Displays a summary of the hyperparameter search space, providing an overview of the hyperparameters being tuned.



### Exercise 4: Running the hyperparameter search 

In this exercise, you run the hyperparameter search using the `search` method of the tuner. You provide the training and validation data along with the number of epochs. After the search is complete, the results summary displays the best hyperparameter configurations found. 

**Run the search:**
- Use the `search` method of the tuner. 
- Pass in the training data, validation data, and the number of epochs


In [9]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary() 

Trial 10 Complete [00h 01m 31s]
val_accuracy: 0.9682999849319458

Best val_accuracy So Far: 0.9800999760627747
Total elapsed time: 00h 30m 51s
Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
units: 416
learning_rate: 0.0011252601260271773
Score: 0.9800999760627747

Trial 07 summary
Hyperparameters:
units: 416
learning_rate: 0.0007598433863293641
Score: 0.979750007390976

Trial 01 summary
Hyperparameters:
units: 352
learning_rate: 0.0011779860142648606
Score: 0.9794000089168549

Trial 00 summary
Hyperparameters:
units: 224
learning_rate: 0.0004980947137951163
Score: 0.9788500070571899

Trial 06 summary
Hyperparameters:
units: 288
learning_rate: 0.002964232517209841
Score: 0.9769499897956848

Trial 08 summary
Hyperparameters:
units: 384
learning_rate: 0.005820326357694492
Score: 0.9713999927043915

Trial 09 summary
Hyperparameters:
units: 96
learning_rate: 0.005211918105655485
Score: 0

#### Explanation

This command runs the hyperparameter search:

- **`epochs=5`**: Each trial is trained for 5 epochs.
- **`validation_data=(x_val, y_val)`**: The validation data to evaluate the model's performance during the search.

After the search is complete, this command displays a summary of the best hyperparameter configurations found during the search.


## Exercise 5: Analyzing and using the best hyperparameters 

In this exercise, you retrieve the best hyperparameters found during the search and print their values. You then build a model with these optimized hyperparameters and train it on the full training data set. Finally, you evaluate the model’s performance on the test set to ensure that it performs well with the selected hyperparameters. 

**Retrieve the best hyperparameters:**
- Use the `get_best_hyperparameters` method to get the best hyperparameters. 
- Print the optimal values for the hyperparameters. 

**Build and train the model:**
- Build a model using the best hyperparameters. 
- Train the model on the full training data set and evaluate its performance on the test set.


In [10]:
# Step 1: Retrieve the best hyperparameters 

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

# Step 2: Build and Train the Model with Best Hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 

 

The optimal number of units in the first dense layer is 416. 

The optimal learning rate for the optimizer is 0.0011252601260271773. 


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.8838 - loss: 0.3831 - val_accuracy: 0.9648 - val_loss: 0.1197
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9704 - loss: 0.0970 - val_accuracy: 0.9724 - val_loss: 0.0915
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9845 - loss: 0.0539 - val_accuracy: 0.9721 - val_loss: 0.0956
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9879 - loss: 0.0383 - val_accuracy: 0.9747 - val_loss: 0.0882
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.9902 - loss: 0.0300 - val_accuracy: 0.9758 - val_loss: 0.0811
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9939 - loss: 0.0209 - val_accuracy: 0.9754 - val_loss: 0.0882
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0

In [ ]:
type(best_hps)

#### Explanation

This code retrieves the best hyperparameters found during the search:

- **`get_best_hyperparameters(num_trials=1)`**: Gets the best hyperparameter configuration.
- **`print(f"...")`**: Prints the best hyperparameters.
- **`model.fit(...)`**: Trains the model on the full training data with a validation split of 20%.
- **`model.evaluate(...)`**: Evaluates the model on the test (validation) dataset and prints the accuracy, which gives an indication of how well the model generalizes.


## Practice exercises 

### Exercise 1: Setting Up Keras Tuner 

#### Objective: 
Learn how to set up Keras Tuner and prepare the environment for hyperparameter tuning. 

#### Instructions: 
1. Install Keras Tuner.
2. Import necessary libraries.
3. Load and preprocess the MNIST data set.


In [11]:
# Write your code here
!pip install keras-tuner


In [12]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

In [13]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

In [14]:
x_train.shape, x_val.shape

((60000, 28, 28), (10000, 28, 28))

<details>
    <summary>Click here for Solution</summary>

```python
!pip install keras-tuner 

# Step 2: Import necessary libraries 
import keras_tuner as kt 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.optimizers import Adam 

# Step 3: Load and preprocess the MNIST data set 
(x_train, y_train), (x_val, y_val) = mnist.load_data() 
x_train, x_val = x_train / 255.0, x_val / 255.0 

# Print the shapes of the training and validation datasets
print(f'Training data shape: {x_train.shape}') 
print(f'Validation data shape: {x_val.shape}')
```

</details>


### Exercise 2: Defining the model with hyperparameters 

#### Objective: 
Define a model-building function that uses hyperparameters to configure the model architecture. 

#### Instructions: 
1. Define a model-building function that uses the `HyperParameters` object to specify the number of units in a dense layer and the learning rate. 
2. Compile the model with sparse categorical cross-entropy loss and Adam optimizer. 


In [36]:
# Write your code here
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step = 32), activation = 'relu'),
        Dense(units=hp.Int('units1', min_value=32, max_value=512, step = 32), activation = 'relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value = 1e-4,  max_value=1e-2, sampling= 'LOG')),
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy']
                 )

    return model

<details>
    <summary>Click here for Solution</summary>

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

# Step 1: Define a model-building function
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
```

</details>


### Exercise 3: Configuring the hyperparameter search 

#### Objective: 
Set up Keras Tuner to search for the best hyperparameter configuration. 

#### Instructions: 
1. Create a `RandomSearch` tuner using the model-building function. 
2. Specify the optimization objective, number of trials, and directory for storing results.


In [37]:
# Write your code here
tuner1 = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='ex1_dir',
    project_name = 'ex1_to_kt'
)

tuner1.search_space_summary()

Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


<details>
    <summary>Click here for Solution</summary>

```python
import keras_tuner as kt

# Step 1: Create a RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,  # Ensure 'build_model' function is defined from previous code
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()
```

</details>


### Exercise 4: Running the hyperparameter search

#### Objective: 
Run the hyperparameter search and dispaly the summary of the results.

#### Instructions: 
1. Run the hyperparameter search using the `search` method of the tuner. 
2. Pass in the training data, validation data, and the number of epochs. 
3. Display a summary of the results. 


In [38]:
# Write your code here
tuner1.search(x_train, y_train, epochs=5, validation_data = (x_val, y_val))

tuner1.results_summary()

Trial 5 Complete [00h 05m 55s]
val_accuracy: 0.9796999990940094

Best val_accuracy So Far: 0.9796999990940094
Total elapsed time: 00h 20m 56s
Results summary
Results in ex1_dir/ex1_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
units: 480
units1: 288
learning_rate: 0.0007370553076887802
Score: 0.9796999990940094

Trial 1 summary
Hyperparameters:
units: 128
units1: 288
learning_rate: 0.0026120407158734663
Score: 0.976500004529953

Trial 3 summary
Hyperparameters:
units: 96
units1: 96
learning_rate: 0.002372999167933176
Score: 0.9749999940395355

Trial 2 summary
Hyperparameters:
units: 480
units1: 288
learning_rate: 0.004082687674516097
Score: 0.9735499918460846

Trial 0 summary
Hyperparameters:
units: 448
units1: 192
learning_rate: 0.0032580468282681196
Score: 0.9733999967575073


<details>
    <summary>Click here for Solution</summary>

```python
# Step 1: Run the hyperparameter search 

tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

 # Display a summary of the results 

tuner.results_summary()
```

</details>


### Exercise 5: Analyzing and using the best hyperparameters 

#### Objective: 
Retrieve the best hyperparameters from the search and build a model with these optimized values. 

#### Instructions: 
1. Retrieve the best hyperparameters using the `get_best_hyperparameters` method. 
2. Build a model using the best hyperparameters. 
3. Train the model on the full training data set and evaluate its performance on the validation set.


In [42]:
best_hps = tuner1.get_best_hyperparameters(num_trials=1)[0]
print(f" units: {best_hps.get('units')}, units1: {best_hps.get('units1')}, Learning rate: {best_hps.get('learning_rate')}")


 units: 480, units1: 288, Learning rate: 0.0007370553076887802


In [41]:
# Write your code here
best_hps = tuner1.get_best_hyperparameters(num_trials=1)[0]
#print(f" units: {best_hps.get('units')}, Learning rate: {best_hps.get('learning_rate')}")

model = tuner1.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split = 0.2)

val_loss, val_acc = model.evaluate(x_val, y_val)
print(f'val accuracy: {val_acc}')

 units: 480, Learning rate: 0.0007370553076887802
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.8924 - loss: 0.3707 - val_accuracy: 0.9663 - val_loss: 0.1109
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.9749 - loss: 0.0834 - val_accuracy: 0.9686 - val_loss: 0.1011
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.9838 - loss: 0.0515 - val_accuracy: 0.9730 - val_loss: 0.0964
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.9879 - loss: 0.0374 - val_accuracy: 0.9747 - val_loss: 0.0911
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.9906 - loss: 0.0285 - val_accuracy: 0.9776 - val_loss: 0.0922
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.9912 - loss: 0.0244 - val_accuracy: 0.9737 - val_loss: 0.1002
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.9941 - loss: 0.0177 - val_accuracy: 0.9768 - val_loss: 0.1018
Epoch 8/10
1500/1500 ━━━━━

<details>
    <summary>Click here for Solution</summary>

```python
# Step 1: Retrieve the best hyperparameters 

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 

print(f""" 

The optimal number of units in the first dense layer is {best_hps.get('units')}. 

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 

""") 

 # Step 2: Build and train the model with best hyperparameters 

model = tuner.hypermodel.build(best_hps) 

model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

 # Evaluate the model on the validation set 

val_loss, val_acc = model.evaluate(x_val, y_val) 

print(f'Validation accuracy: {val_acc}') 
```

</details>


### Conclusion 

Congratulations on completing this lab! You have learned to set up Keras Tuner and prepare the environment for hyperparameter tuning. In addition, you defined a model-building function that uses hyperparameters to configure the model architecture. You configured Keras Tuner to search for the best hyperparameter configuration and learned to run the hyperparameter search and analyze the results. Finally, you retrieved the best hyperparameters and built a model with these optimized values. 


## Authors


Skillup


Copyright ©IBM Corporation. All rights reserved.
